# Machine Learning Homework 2

## Naive Bayes Classifier

In [328]:
import numpy as np

In [329]:
# parse the input files

import struct

with open('./train-labels-idx1-ubyte', 'rb') as f:
    data = f.read()
    offset = 0
    fmt = '>2i'
    magic_num, num_labels = struct.unpack_from(fmt, data, offset)
    offset += struct.calcsize(fmt)
    fmt = '>{}B'.format(str(num_labels))
    train_labels = np.array(struct.unpack_from(fmt, data, offset))

with open('./train-images-idx3-ubyte', 'rb') as f:
    data = f.read()
    offset = 0
    fmt = '>4i'
    magic_num, num_images, height, width = struct.unpack_from(fmt, data, offset)
    image_size = height*width
    offset += struct.calcsize(fmt)
    fmt = '>{}B'.format(str(image_size))
    train_images = np.empty((num_images, image_size))
    for i in range(num_images):
        train_images[i] = np.array(struct.unpack_from(fmt, data, offset))
        offset += struct.calcsize(fmt)

with open('./t10k-labels-idx1-ubyte', 'rb') as f:
    data = f.read()
    offset = 0
    fmt = '>2i'
    magic_num, num_labels = struct.unpack_from(fmt, data, offset)
    offset += struct.calcsize(fmt)
    fmt = '>{}B'.format(str(num_labels))
    test_labels = struct.unpack_from(fmt, data, offset)

with open('./t10k-images-idx3-ubyte', 'rb') as f:
    data = f.read()
    offset = 0
    fmt = '>4i'
    magic_num, num_images, height, width = struct.unpack_from(fmt, data, offset)
    images_size = height*width
    offset += struct.calcsize(fmt)
    fmt = '>{}B'.format(str(image_size))
    test_images = np.empty((num_images, image_size))
    for i in range(num_images):
        test_images[i] = np.array(struct.unpack_from(fmt, data, offset))
        offset += struct.calcsize(fmt)

In [353]:
num_images = 60000
height, width, size = 28, 28, 784
mode = 0

if mode == 0:
    # training
    unique, count = np.unique(train_labels, return_counts=True)
    counts = dict(zip(unique, count))
    bins = np.ones((10, size, 32))
    for i in range(len(train_images)):
        for j in range(size):
            bins[train_labels[i], j, int(train_images[i, j]/8)] += 1
    for i in range(10):
        bins[i] = np.array(bins[i]/(count[i]+32))

    # testing
    d = {True: 0, False: 0}
    for i in range(len(test_images)):
        p = np.zeros(10)
        img = test_images[i]
        data = (img/8).astype(int)
        for j in range(10):
            p[j] = np.log([bins[j, k, data[k]] for k in range(size)]).sum() + np.log(counts[j]/num_images)
        result = np.argmax(p)
        p = p/np.sum(p)
        is_match = result==test_labels[i]
        d[is_match] = d[is_match] + 1
        
        print('Posterior (in log scale):')
        for k, l in enumerate(p):
            print('%d: %f' % (k, l))
        print('Prediction: %d, Ans: %d' % (result, test_labels[i]))
        print()
    print('Error rate: %f' % (d[False]/(d[True]+d[False])))
else:
    # training
    unique, count = np.unique(train_labels, return_counts=True)
    counts = dict(zip(unique, count))
    mean = np.empty((10, size))
    variance = np.empty((10, size))
    for i in range(10):
        mean[i] = np.mean(train_images[train_labels==i], axis=0)
        variance[i] = np.var(train_images[train_labels==i], axis=0)
    
    # testing
    d = {True: 0, False: 0}
    for i in range(len(test_images)):
        p = np.zeros(10)
        img = test_images[i]
        for j in range(10):
            for k in range(size):
                if variance[j, k] != 0:
                    p[j] += (-0.39908993417 - np.log(variance[j, k]**0.5) + (((img[k]-mean[j, k])**2)/(-2*variance[j, k])))
                else:
                    p[j] += (-0.39908993417 - 3)
            p[j] += np.log(counts[j]/num_images)
        result = np.argmax(p)
        p = p/np.sum(p)
        is_match = result==test_labels[i]
        d[is_match] = d[is_match] + 1
        
        print('Posterior (in log scale):')
        for k, l in enumerate(p):
            print('%d: %f' % (k, l))
        print('Prediction: %d, Ans: %d' % (result, test_labels[i]))
        print()
    print('Error rate: %f' % (d[False]/(d[True]+d[False])))
    
mean = np.empty((10, size))
for i in range(10):
    mean[i] = np.mean(train_images[train_labels==i], axis=0)
imagination = np.empty((10, size))
for i in range(10):
    imagination[i] = np.where(mean[i]>=128, 1, 0)
imagination = imagination.astype(int)
print('Imagination of numbers in Baysian classifier:')
print()
for i in range(10):
    print('%d:' % i)
    for j in range(height):
        print(str(imagination[i, j*width:(j+1)*width]))

Posterior (in log scale):
0: 0.111155
1: 0.117633
2: 0.105172
3: 0.100138
4: 0.093892
5: 0.097558
6: 0.114420
7: 0.074442
8: 0.099521
9: 0.086069
Prediction: 7, Ans: 7

Posterior (in log scale):
0: 0.100200
1: 0.107035
2: 0.083330
3: 0.090346
4: 0.108774
5: 0.092526
6: 0.089659
7: 0.118930
8: 0.096340
9: 0.112860
Prediction: 2, Ans: 2

Posterior (in log scale):
0: 0.128477
1: 0.063865
2: 0.098681
3: 0.100613
4: 0.103425
5: 0.096712
6: 0.103354
7: 0.101449
8: 0.100410
9: 0.103014
Prediction: 1, Ans: 1

Posterior (in log scale):
0: 0.077796
1: 0.140867
2: 0.097081
3: 0.096539
4: 0.101507
5: 0.092703
6: 0.090877
7: 0.102680
8: 0.097816
9: 0.102134
Prediction: 0, Ans: 0

Posterior (in log scale):
0: 0.104334
1: 0.131132
2: 0.096731
3: 0.105358
4: 0.082065
5: 0.097407
6: 0.098982
7: 0.095260
8: 0.099180
9: 0.089551
Prediction: 4, Ans: 4

Posterior (in log scale):
0: 0.132870
1: 0.061897
2: 0.099934
3: 0.099554
4: 0.103247
5: 0.097384
6: 0.110099
7: 0.099316
8: 0.095703
9: 0.099995
Predictio

Posterior (in log scale):
0: 0.100698
1: 0.116632
2: 0.106519
3: 0.098363
4: 0.093470
5: 0.086960
6: 0.105773
7: 0.101070
8: 0.095301
9: 0.095212
Prediction: 5, Ans: 5

Posterior (in log scale):
0: 0.098701
1: 0.117444
2: 0.100374
3: 0.090265
4: 0.097240
5: 0.090568
6: 0.099599
7: 0.105088
8: 0.098099
9: 0.102623
Prediction: 3, Ans: 5

Posterior (in log scale):
0: 0.096851
1: 0.122888
2: 0.087874
3: 0.099879
4: 0.095386
5: 0.099049
6: 0.089929
7: 0.105553
8: 0.098333
9: 0.104258
Prediction: 2, Ans: 6

Posterior (in log scale):
0: 0.090291
1: 0.131048
2: 0.101377
3: 0.092418
4: 0.100036
5: 0.089784
6: 0.100259
7: 0.106858
8: 0.089641
9: 0.098289
Prediction: 8, Ans: 0

Posterior (in log scale):
0: 0.100966
1: 0.142951
2: 0.097287
3: 0.100303
4: 0.079959
5: 0.094391
6: 0.099162
7: 0.097255
8: 0.096913
9: 0.090812
Prediction: 4, Ans: 4

Posterior (in log scale):
0: 0.130542
1: 0.061891
2: 0.101073
3: 0.099777
4: 0.104386
5: 0.096096
6: 0.108503
7: 0.098281
8: 0.099702
9: 0.099751
Predictio

Posterior (in log scale):
0: 0.111230
1: 0.104886
2: 0.100269
3: 0.095790
4: 0.099781
5: 0.092872
6: 0.110010
7: 0.104557
8: 0.082821
9: 0.097784
Prediction: 8, Ans: 8

Posterior (in log scale):
0: 0.130250
1: 0.063452
2: 0.100241
3: 0.096575
4: 0.108767
5: 0.095972
6: 0.101744
7: 0.105155
8: 0.095229
9: 0.102615
Prediction: 1, Ans: 1

Posterior (in log scale):
0: 0.087243
1: 0.130011
2: 0.096029
3: 0.093095
4: 0.100277
5: 0.090365
6: 0.095975
7: 0.109938
8: 0.093301
9: 0.103766
Prediction: 0, Ans: 0

Posterior (in log scale):
0: 0.129336
1: 0.065134
2: 0.093885
3: 0.096625
4: 0.108142
5: 0.097057
6: 0.105971
7: 0.108441
8: 0.089558
9: 0.105849
Prediction: 1, Ans: 1

Posterior (in log scale):
0: 0.098305
1: 0.130345
2: 0.092276
3: 0.102120
4: 0.093168
5: 0.098137
6: 0.079145
7: 0.105131
8: 0.100010
9: 0.101365
Prediction: 6, Ans: 6

Posterior (in log scale):
0: 0.110049
1: 0.123652
2: 0.099940
3: 0.111299
4: 0.084948
5: 0.097141
6: 0.100932
7: 0.091709
8: 0.092177
9: 0.088153
Predictio

Posterior (in log scale):
0: 0.103386
1: 0.130608
2: 0.102020
3: 0.100714
4: 0.082488
5: 0.094259
6: 0.103374
7: 0.092398
8: 0.101380
9: 0.089373
Prediction: 4, Ans: 4

Posterior (in log scale):
0: 0.105904
1: 0.102397
2: 0.107050
3: 0.100824
4: 0.096320
5: 0.091631
6: 0.113848
7: 0.093276
8: 0.095387
9: 0.093363
Prediction: 5, Ans: 5

Posterior (in log scale):
0: 0.110737
1: 0.132413
2: 0.100854
3: 0.096052
4: 0.086868
5: 0.094162
6: 0.101253
7: 0.094604
8: 0.098519
9: 0.084538
Prediction: 9, Ans: 9

Posterior (in log scale):
0: 0.124667
1: 0.086211
2: 0.100868
3: 0.091628
4: 0.100492
5: 0.095767
6: 0.111097
7: 0.098168
8: 0.095621
9: 0.095483
Prediction: 1, Ans: 3

Posterior (in log scale):
0: 0.120172
1: 0.126575
2: 0.106723
3: 0.098269
4: 0.086054
5: 0.094983
6: 0.106756
7: 0.090704
8: 0.092462
9: 0.077301
Prediction: 9, Ans: 9

Posterior (in log scale):
0: 0.088565
1: 0.135759
2: 0.094833
3: 0.095553
4: 0.097485
5: 0.092418
6: 0.097368
7: 0.105662
8: 0.093785
9: 0.098574
Predictio

Posterior (in log scale):
0: 0.095155
1: 0.106121
2: 0.084163
3: 0.087127
4: 0.108371
5: 0.090882
6: 0.096076
7: 0.123233
8: 0.094113
9: 0.114760
Prediction: 2, Ans: 2

Posterior (in log scale):
0: 0.112427
1: 0.129625
2: 0.101725
3: 0.103847
4: 0.087051
5: 0.096682
6: 0.102544
7: 0.089220
8: 0.096938
9: 0.079941
Prediction: 9, Ans: 9

Posterior (in log scale):
0: 0.115770
1: 0.111254
2: 0.083008
3: 0.096831
4: 0.096542
5: 0.099165
6: 0.093243
7: 0.111213
8: 0.094441
9: 0.098532
Prediction: 2, Ans: 2

Posterior (in log scale):
0: 0.084956
1: 0.130014
2: 0.099195
3: 0.091566
4: 0.101494
5: 0.088703
6: 0.104265
7: 0.106478
8: 0.093326
9: 0.100004
Prediction: 0, Ans: 0

Posterior (in log scale):
0: 0.113014
1: 0.130085
2: 0.101963
3: 0.092878
4: 0.084676
5: 0.092104
6: 0.102244
7: 0.099790
8: 0.098348
9: 0.084897
Prediction: 4, Ans: 4

Posterior (in log scale):
0: 0.067820
1: 0.139277
2: 0.091510
3: 0.094744
4: 0.103235
5: 0.089592
6: 0.100462
7: 0.109505
8: 0.097483
9: 0.106372
Predictio

...

Posterior (in log scale):
0: 0.101378
1: 0.119768
2: 0.090767
3: 0.078285
4: 0.102806
5: 0.090020
6: 0.104506
7: 0.115331
8: 0.090191
9: 0.106948
Prediction: 3, Ans: 3

Posterior (in log scale):
0: 0.117533
1: 0.107720
2: 0.098817
3: 0.104210
4: 0.088783
5: 0.095389
6: 0.101115
7: 0.101180
8: 0.091316
9: 0.093938
Prediction: 4, Ans: 4

Posterior (in log scale):
0: 0.113955
1: 0.100387
2: 0.097612
3: 0.101105
4: 0.095123
5: 0.090482
6: 0.099726
7: 0.104775
8: 0.095290
9: 0.101545
Prediction: 5, Ans: 5

Posterior (in log scale):
0: 0.087145
1: 0.132792
2: 0.090790
3: 0.099734
4: 0.097642
5: 0.093293
6: 0.080049
7: 0.118445
8: 0.094334
9: 0.105778
Prediction: 6, Ans: 6

Posterior (in log scale):
0: 0.111688
1: 0.108021
2: 0.104866
3: 0.105288
4: 0.094864
5: 0.096112
6: 0.110830
7: 0.084920
8: 0.094367
9: 0.089043
Prediction: 7, Ans: 7

Posterior (in log scale):
0: 0.105324
1: 0.095004
2: 0.097961
3: 0.097068
4: 0.102137
5: 0.088569
6: 0.113868
7: 0.108063
8: 0.086537
9: 0.105469
Predictio

Posterior (in log scale):
0: 0.109688
1: 0.099047
2: 0.094364
3: 0.100947
4: 0.101382
5: 0.095330
6: 0.108398
7: 0.100457
8: 0.090611
9: 0.099775
Prediction: 8, Ans: 8

Posterior (in log scale):
0: 0.103610
1: 0.114821
2: 0.102306
3: 0.106367
4: 0.094607
5: 0.093504
6: 0.106872
7: 0.089301
8: 0.094707
9: 0.093904
Prediction: 7, Ans: 7

Posterior (in log scale):
0: 0.097635
1: 0.115673
2: 0.094557
3: 0.106116
4: 0.095123
5: 0.090909
6: 0.089101
7: 0.110299
8: 0.098035
9: 0.102550
Prediction: 6, Ans: 6

Posterior (in log scale):
0: 0.114876
1: 0.109949
2: 0.097784
3: 0.103481
4: 0.088967
5: 0.095810
6: 0.101932
7: 0.097946
8: 0.095560
9: 0.093695
Prediction: 4, Ans: 4

Posterior (in log scale):
0: 0.123436
1: 0.101915
2: 0.107101
3: 0.102019
4: 0.092178
5: 0.096257
6: 0.110275
7: 0.088372
8: 0.093726
9: 0.084721
Prediction: 9, Ans: 9

Posterior (in log scale):
0: 0.114628
1: 0.110169
2: 0.106606
3: 0.102702
4: 0.091526
5: 0.094350
6: 0.109298
7: 0.093350
8: 0.092331
9: 0.085039
Predictio

Posterior (in log scale):
0: 0.086403
1: 0.122774
2: 0.098650
3: 0.095989
4: 0.100937
5: 0.087327
6: 0.103563
7: 0.106209
8: 0.094392
9: 0.103757
Prediction: 0, Ans: 0

Posterior (in log scale):
0: 0.100909
1: 0.129594
2: 0.095045
3: 0.110789
4: 0.082011
5: 0.096512
6: 0.096345
7: 0.094501
8: 0.100680
9: 0.093614
Prediction: 4, Ans: 4

Posterior (in log scale):
0: 0.103422
1: 0.107332
2: 0.098482
3: 0.098973
4: 0.100469
5: 0.092750
6: 0.114167
7: 0.097557
8: 0.087804
9: 0.099044
Prediction: 8, Ans: 2

Posterior (in log scale):
0: 0.119036
1: 0.102444
2: 0.104771
3: 0.105732
4: 0.095932
5: 0.097815
6: 0.112307
7: 0.078080
8: 0.098737
9: 0.085145
Prediction: 7, Ans: 7

Posterior (in log scale):
0: 0.114783
1: 0.110505
2: 0.106931
3: 0.105904
4: 0.093570
5: 0.098271
6: 0.114090
7: 0.073542
8: 0.098299
9: 0.084105
Prediction: 7, Ans: 7

Posterior (in log scale):
0: 0.103910
1: 0.094668
2: 0.105137
3: 0.096220
4: 0.104958
5: 0.084826
6: 0.111742
7: 0.108105
8: 0.087144
9: 0.103289
Predictio

Posterior (in log scale):
0: 0.093147
1: 0.134268
2: 0.093128
3: 0.104678
4: 0.087274
5: 0.095202
6: 0.088786
7: 0.102703
8: 0.101158
9: 0.099657
Prediction: 4, Ans: 4

Posterior (in log scale):
0: 0.080851
1: 0.121163
2: 0.097331
3: 0.094608
4: 0.102787
5: 0.088115
6: 0.103458
7: 0.108432
8: 0.096276
9: 0.106977
Prediction: 0, Ans: 0

Posterior (in log scale):
0: 0.091634
1: 0.131478
2: 0.092995
3: 0.104586
4: 0.095880
5: 0.096552
6: 0.079915
7: 0.106070
8: 0.100110
9: 0.100781
Prediction: 6, Ans: 6

Posterior (in log scale):
0: 0.124852
1: 0.070383
2: 0.096070
3: 0.098441
4: 0.102279
5: 0.094551
6: 0.112125
7: 0.104027
8: 0.092999
9: 0.104273
Prediction: 1, Ans: 1

Posterior (in log scale):
0: 0.087334
1: 0.121382
2: 0.099491
3: 0.095533
4: 0.099080
5: 0.088779
6: 0.106871
7: 0.101710
8: 0.097651
9: 0.102168
Prediction: 0, Ans: 0

Posterior (in log scale):
0: 0.075273
1: 0.132168
2: 0.098782
3: 0.097120
4: 0.101336
5: 0.086499
6: 0.099071
7: 0.110239
8: 0.093675
9: 0.105835
Predictio

[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 0 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 0 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0

## Online Learning

In [201]:
a, b = 10, 1

with open('./test.txt', 'r') as f:
    data = f.read().split('\n')

In [202]:
def factorial(end, start=0):
    if end == 0:
        return 1
    elif end == start:
        return end
    else:
        return end * factorial(end-1, start)
    
def combination(n, m):
    return int(factorial(n, n-m+1)/factorial(m))

def likelihood(a, b):
    p = a/(a+b)
    return combination(a+b, a) * (p**a) * ((1-p)**b)

def parse_input(s):
    return (s.count('1'), s.count('0'))

In [203]:
posterior = (a, b)
for i in range(len(data)):
    prior = (posterior[0], posterior[1])
    new_input = parse_input(data[i])
    posterior = (prior[0]+new_input[0], prior[1]+new_input[1])
    l = likelihood(new_input[0], new_input[1])
    
    print('case %d: %s' % (i, data[i]))
    print('Likelihood: %f' % l)
    print('Beta prior:     a = %d   b = %d' % (prior[0], prior[1]))
    print('Beta posterior: a = %d   b = %d' % (posterior[0], posterior[1]))
    print()

case 0: 0101010101001011010101
Likelihood: 0.168188
Beta prior:     a = 10   b = 1
Beta posterior: a = 21   b = 12

case 1: 0110101
Likelihood: 0.293755
Beta prior:     a = 21   b = 12
Beta posterior: a = 25   b = 15

case 2: 010110101101
Likelihood: 0.228605
Beta prior:     a = 25   b = 15
Beta posterior: a = 32   b = 20

case 3: 0101101011101011010
Likelihood: 0.182869
Beta prior:     a = 32   b = 20
Beta posterior: a = 43   b = 28

case 4: 111101100011110
Likelihood: 0.214307
Beta prior:     a = 43   b = 28
Beta posterior: a = 53   b = 33

case 5: 101110111000110
Likelihood: 0.206598
Beta prior:     a = 53   b = 33
Beta posterior: a = 62   b = 39

case 6: 1010010111
Likelihood: 0.250823
Beta prior:     a = 62   b = 39
Beta posterior: a = 68   b = 43

case 7: 11101110110
Likelihood: 0.261968
Beta prior:     a = 68   b = 43
Beta posterior: a = 76   b = 46

case 8: 01000111101
Likelihood: 0.236091
Beta prior:     a = 76   b = 46
Beta posterior: a = 82   b = 51

case 9: 110100111
Likeli